In [1]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [10]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other) #This enables addition of integer to the object by converting the integer to an object
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out

  def __pow__(self, other):
     assert isinstance(other, (int, float)), "only supporting int/float powers for now"
     out = Value(self.data**other, (self,), f'**{other}')

     def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
     out._backward = _backward

     return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  
  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [11]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [12]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.backward()

In [ ]:
draw_dot(o)

In [13]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
# ----
e = (2*n).exp()
o = (e - 1) / (e + 1)
# ----
o.label = 'o'
o.backward()
draw_dot(o)

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [14]:
import torch
import random

In [15]:
x1 = torch.Tensor([2.0]).double(); x1.requires_grad = True
w1 = torch.Tensor([-3.0]).double(); w1.requires_grad = True
x2 = torch.Tensor([0.0]).double(); x2.requires_grad = True
w2 = torch.Tensor([1.0]).double(); w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double(); b.requires_grad = True
n = w1*x1 + w2*x2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()
print("-----")
print('x1 ', x1.grad.item())
print('w1 ', w1.grad.item())
print('x2 ', x2.grad.item())
print('w2 ', w2.grad.item())

0.7071066904050358
-----
x1  -1.5000003851533106
w1  1.0000002567688737
x2  0.5000001283844369
w2  0.0


In [88]:
class Neuron:
    def __init__(self, nin): #nin is the number of input parameters
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w,x)), self.b) #zipping joins wi corresponding to xi matching their indices
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b] #The 3 parameters functions are there to print a list of parameters

class Layer:
  
  def __init__(self, nin, nout): #nin is number of neurons present in input for that Layer, nout is number of neurons present in output for that Layer
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs #this is to change the output from a list of Value items to a Value Item if there is only 1 output

  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()] #The 3 parameters functions are there to print a list of parameters


class MLP:
  
  def __init__(self, nin, nouts): #Here nouts is a list containing the number of output nodes required in each layer
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
      
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()] #The 3 parameters functions are there to print a list of parameters


In [149]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x) #this indirectly calls the __call__ function

Value(data=-0.49001212647111614)

In [150]:
n.parameters()

[Value(data=0.9951383438872456),
 Value(data=-0.3531143249006419),
 Value(data=0.32861575445402624),
 Value(data=-0.2994068221786059),
 Value(data=-0.04824605906082735),
 Value(data=0.1940483616230242),
 Value(data=-0.017859152827122138),
 Value(data=-0.20472914593538616),
 Value(data=0.3519670821769785),
 Value(data=-0.7143790106668368),
 Value(data=0.0786157265947165),
 Value(data=-0.762536635863647),
 Value(data=0.33879699654530326),
 Value(data=-0.3903964868380658),
 Value(data=0.33190517888357096),
 Value(data=-0.9385969825701526),
 Value(data=-0.47003700768807066),
 Value(data=-0.738091621807224),
 Value(data=-0.21152480663064877),
 Value(data=-0.7922860954341964),
 Value(data=-0.9251633834023198),
 Value(data=0.5327834381327385),
 Value(data=0.04637072103967155),
 Value(data=0.6693752977487928),
 Value(data=-0.4573399222201968),
 Value(data=-0.23554566933169152),
 Value(data=0.6404257770281461),
 Value(data=0.2563861572912598),
 Value(data=0.11055411929918302),
 Value(data=0.927

In [151]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [152]:
for k in range(20):
    #forward pass
    ypred = [n(x) for x in xs]
    loss = sum((Value(ygt) - yout)**2 for ygt, yout in zip(ys, ypred))
    #backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    #update
    for p in n.parameters():
        p.data += -0.05 * p.grad
    print(k, loss.data)

0 9.016551825899949
1 6.108055078083197
2 4.757767197858312
3 3.868842434095181
4 3.1601145744116312
5 2.3625604510929823
6 1.5561478204885892
7 0.9788647415919043
8 0.6448116920752289
9 0.45454688901683155
10 0.34013511918812606
11 0.2666853267660303
12 0.21670088358972617
13 0.181007604046987
14 0.1545033539627531
15 0.1341855127089691
16 0.11819640659746153
17 0.10533608101135157
18 0.09480023445701193
19 0.08603235187753928
